In [86]:
import jieba
import pandas as pd
from collections import defaultdict
import math
import operator

jieba.initialize()

# 最简单的TF-IDF分类算法

In [87]:
def load_abstract():
    train_df_iter = pd.read_csv('datasets/train_dataset.csv', sep='|')
    results = []
    for i in train_df_iter['abstract'].tolist():

        target = jieba.cut(i, HMM=False)
        results.append([i for i in target])
    return results

def feature_select(list_words):
    # 总词频统计
    doc_frequency = defaultdict(int)
    for word_list in list_words:
        for i in word_list:
            doc_frequency[i] += 1

    # 计算每个词的TF值
    word_tf = {}  # 存储没个词的tf值
    for i in doc_frequency:
        word_tf[i] = doc_frequency[i] / sum(doc_frequency.values())

    # 计算每个词的IDF值
    doc_num = len(list_words)
    word_idf = {}  # 存储每个词的idf值
    word_doc = defaultdict(int)  # 存储包含该词的文档数
    for i in doc_frequency:
        for j in list_words:
            if i in j:
                word_doc[i] += 1
    for i in doc_frequency:
        word_idf[i] = math.log(doc_num / (word_doc[i] + 1))

    # 计算每个词的TF*IDF的值
    word_tf_idf = {}
    for i in doc_frequency:
        word_tf_idf[i] = word_tf[i] * word_idf[i]

    # 对字典按值由大到小排序
    dict_feature_select = sorted(word_tf_idf.items(), key=operator.itemgetter(1), reverse=True)
    return dict_feature_select


data_list = load_abstract()  # 加载数据
features = feature_select(data_list)  # 所有词的TF-IDF值
print(len(features))

with open('datasets/tf_idf.text', 'w', encoding='UTF-8') as writer:
    writer.writelines('\n'.join([f'{i[0]} {i[1]*1000}' for i in features]))

11594


In [134]:
import jieba.analyse
import pandas as pd
jieba.analyse.set_idf_path("./datasets/tf_idf.text")

def find_key_words(row):
    return jieba.analyse.textrank(row.encode('utf-8'), topK=5, withWeight=False, allowPOS=('ns', 'n', 'vn', 'v'))

train_df['textrank'] = train_df['abstract'].apply(find_key_words)
train_df = train_df.rename(columns={'keys':'high_key'})

train_df = train_df[~(train_df["abstract"].str.contains('\t\t')&train_df["abstract"].str.contains(':'))]
train_df = train_df[~(train_df["abstract"].str.contains('1、')&train_df["abstract"].str.contains(':'))]

def find_key_words(row):
    result = []
    for i in ['限速', '骚扰', '前期', '信号', '短信', '基站', '额度', '流量', '前期', '减免']:
        if i in row['abstract']:
            result.append(i)
    return result

train_df['own_key'] = train_df.apply(find_key_words,axis=1)
train_df.to_csv('datasets/train_dataset_key.csv', index=False, sep='|')

In [ ]:
相似的信息
信用额度：透支
中文转数字
带有'\t'得都删除


限速
骚扰
前期
信号
短信
基站
信用额度
流量
前期
减免

所在位置->基站
#rule base
奇怪的abstract的格式，
交了费用但是没有续费成功
